

## Melanoma Classification with Transfer Learning and Hyperparameter Tuning

### Project Overview

This project focuses on the classification of melanoma using dermoscopic images and deep learning techniques. The main objective is to improve the diagnostic performance of a pre-trained convolutional neural network model through **fine-tuning** and **hyperparameter optimization**.

We start with an EfficientNetB0 model pre-trained on ImageNet and perform transfer learning using a custom melanoma dataset. To enhance the model’s performance, we apply Keras Tuner to search for optimal training parameters such as dropout rate and learning rate.

### Dataset

The dataset is structured into three subsets:

* `train/`: Used for training the model.
* `validation/`: Used to tune and validate model performance during training.
* `test/`: Used for final model evaluation.

The dataset is balanced using class weights due to class imbalance (e.g., fewer malignant samples).

### Methodology

1. **Transfer Learning**: Load the EfficientNetB0 model with ImageNet weights.
2. **Fine-Tuning**: All layers are set to be trainable to enable full fine-tuning.
3. **Data Preprocessing**: Images are resized to 224x224 and normalized using `preprocess_input`.
4. **Hyperparameter Tuning**:

   * Performed using `keras-tuner`.
   * Searched over a range of dropout rates and learning rates.




In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
!pip install keras-tuner
import keras_tuner as kt


if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00
Mounted at /content/drive


In [4]:
# !cp -r "/content/drive/My Drive/melonoma_classification_dataset" "/content/dataset"

base_dir = "/content/dataset/"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)

# Calculate weights of the classes balanced.
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))


Found 7953 images belonging to 2 classes.
Found 1897 images belonging to 2 classes.


In [5]:
!cp "/content/drive/My Drive/melonoma_classification_model/efficientnetb0_melanoma_model.keras" "/content/efficientnetb0_melanoma_model.keras"
PRETRAINED_MODEL_PATH = "/content/efficientnetb0_melanoma_model.keras"
model = load_model(PRETRAINED_MODEL_PATH)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 7,534,184 (28.74 MB)

 Trainable params: 1,660,257 (6.33 MB)

 Non-trainable params: 2,553,411 (9.74 MB)

 Optimizer params: 3,320,516 (12.67 MB)

In [6]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, optimizers, Model

def build_model(hp):
    base_model = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3)
    )
    base_model.trainable = True  # Fine-tune tamamı

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(hp.Float("dropout_rate", 0.2, 0.5, step=0.1))(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)

    hp_learning_rate = hp.Choice("lr", values=[1e-4, 1e-5, 5e-5])
    model.compile(
        optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model



In [7]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="/content/drive/My Drive/tuning_results",
    project_name="melanoma_finetune"
)

tuner.search(
    train_generator,
    validation_data=val_generator,
    class_weight=class_weights,
    epochs=10
)


Trial 5 Complete [00h 04m 19s]
val_accuracy: 0.935160756111145

Best val_accuracy So Far: 0.9520295262336731
Total elapsed time: 00h 21m 53s


In [9]:
best_hp = tuner.get_best_hyperparameters(1)[0]

for param, value in best_hp.values.items():
    print(f"  {param}: {value}")

  dropout_rate: 0.4
  lr: 5e-05


In [10]:

# Recompile/build the model with best params.
best_model = build_model(best_hp)

history = best_model.fit(
    train_generator,
    validation_data=val_generator,
    class_weight=class_weights,
    epochs=10
)

best_model.save("/content/drive/My Drive/melonoma_classification_model/final_finetuned_model.keras")


Epoch 1/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 130s 227ms/step - accuracy: 0.7561 - loss: 0.4554 - val_accuracy: 0.9077 - val_loss: 0.2268
Epoch 2/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9172 - loss: 0.1964 - val_accuracy: 0.9304 - val_loss: 0.1586
Epoch 3/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9427 - loss: 0.1467 - val_accuracy: 0.9410 - val_loss: 0.1415
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9500 - loss: 0.1297 - val_accuracy: 0.9446 - val_loss: 0.1380
Epoch 5/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9640 - loss: 0.0931 - val_accuracy: 0.9473 - val_loss: 0.1373
Epoch 6/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9723 - loss: 0.0756 - val_accuracy: 0.9452 - val_loss: 0.1513
Epoch 7/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9820 - loss: 0.0530 - val_accuracy: 0.9483 - val_loss: 0.1507
Epoch 8/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9847 - loss: 0.0462 

### Results

After tuning and evaluating the model, the best performing hyperparameters were found to be:

* `dropout_rate`: **0.4**
* `learning_rate`: **5e-05**

> We saved the model after the finding best suitable params.
